In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
import numpy as np
import datasets

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import sys
sys.path.append('..')
from utils.reasoning import make_segment, split_cot
from torch.nn.utils.rnn import pad_sequence

/home/jovyan/bulatov/envs/env_main/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoModel

from modeling_amt.language_modeling import AssociativeMemoryCell
# from modeling_amt.experimental import AssociativeRecurrentWrapperNoSegmentation

In [5]:
from transformers import PreTrainedModel

In [ ]:
memory_cell_cls = get_cls_by_name(args.memory_cell_cls)
recurrent_wrapper_cls = get_cls_by_name(args.recurrent_wrapper_cls)
base_model = 
logger.info(f'Wrapping in: {memory_cell_cls} and {recurrent_wrapper_cls}')
mem_cell_args = dict(
    base_model=model,
    num_mem_tokens=args.num_mem_tokens,
)
# additional parameters for ARMT model
if args.d_mem is not None:
    mem_cell_args['d_mem'] = args.d_mem
    mem_cell_args['wrap_pos'] = args.wrap_pos
    mem_cell_args['correction'] = not args.no_correction
    # mem_cell_args['use_lora'] = args.use_lora
if args.layers_attr is not None:
    mem_cell_args['layers_attr'] = args.layers_attr
if args.attend_to_previous_input:
    mem_cell_args['attend_to_previous_input'] = args.attend_to_previous_input
cell = memory_cell_cls(**mem_cell_args)
model = recurrent_wrapper_cls(cell,
                                segment_size=args.segment_size,
                                max_n_segments=args.max_n_segments,
                                vary_n_segments=args.vary_n_segments,
                                k2=args.k2,
                                attend_to_previous_input=args.attend_to_previous_input,
                                return_all_logits=False,
                                answer_loss_weight=args.answer_loss_weight
                                )

In [ ]:
from transformers import PretrainedConfig

class ARMTConfig(PretrainedConfig):
    model_type = "armt"

    def __init__(
        self,
        base_model,
        d_mem: int = 32,
        segment_size: int = 128,
        max_n_segments: int = 1,
        vary_n_segments: bool = False,
        k2: int = 1,
        wrap_pos: bool = False,
        correction: bool = True,
        layers_attr: str = None,
        attend_to_previous_input: bool = False,
        return_all_logits: bool = False,
        **kwargs,
    ):
        self.base_model = base_model
        self.d_mem = d_mem
        self.segment_size = segment_size
        self.max_n_segments = max_n_segments
        self.vary_n_segments = vary_n_segments
        self.k2 = k2
        
        super().__init__(**kwargs)
        
        

In [ ]:
class ARMT(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        if args.d_mem is not None:
            mem_cell_args['d_mem'] = args.d_mem
            mem_cell_args['wrap_pos'] = args.wrap_pos
            mem_cell_args['correction'] = not args.no_correction
            # mem_cell_args['use_lora'] = args.use_lora
        if args.layers_attr is not None:
            mem_cell_args['layers_attr'] = args.layers_attr
        if args.attend_to_previous_input:
            mem_cell_args['attend_to_previous_input'] = args.attend_to_previous_input
        cell = memory_cell_cls(**mem_cell_args)
        model = recurrent_wrapper_cls(cell,
                                        segment_size=args.segment_size,
                                        max_n_segments=args.max_n_segments,
                                        vary_n_segments=args.vary_n_segments,
                                        k2=args.k2,
                                        attend_to_previous_input=args.attend_to_previous_input,
                                        return_all_logits=False,
                                        answer_loss_weight=args.answer_loss_weight
                                        )
        

In [10]:
base_model = AutoModelForCausalLM.from_pretrained('gpt2')
memory_cell = AssociativeMemoryCell(base_model, num_mem_tokens=16, d_mem=32, correction=False, wrap_pos=False)
rmt = AssociativeRecurrentWrapper(memory_cell, segment_size=1024)

TypeError: PreTrainedModel.__init__() missing 1 required positional argument: 'config'

In [ ]:

with open(cpt_path, 'rb') as f:
    cpt = torch.load(f)
rmt.load_state_dict(cpt)